![ga4](https://www.google-analytics.com/collect?v=2&tid=G-6VDTYWLKX6&cid=1&en=page_view&sid=1&dl=statmike%2Fvertex-ai-mlops%2FDev%2Fnew&dt=Unified+Content+Holder.ipynb)

# Notes

## Readme.md files

start with header: vertex-ai-mlops/folder/readme.md

all notebooks need a line like this after main header in first cell:

> From the [Working With Document AI](https://github.com/statmike/vertex-ai-mlops/blob/main/Working%20With%20Document%20AI/readme.md) series in the [vertex-ai-mlops](https://github.com/statmike/vertex-ai-mlops/blob/main/readme.md) repository.

---
## Snippets

In [ ]:
import logging

# setup logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.addHandler(logging.StreamHandler())

logging.info('Finished setting up clients')

In [ ]:
try:
    # setting the limit_per_minute to 80, higher than the actual limit of 60
    responses = await study_notes(vocab_words[0:80], limit_per_minute = 80)
except Exception as err:
    print(f"{type(err).__name__} was raised: {err}")

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import vertexai
import vertexai.vision_models # Imagen Models
import vertexai.language_models # PaLM and Codey Models
import vertexai.generative_models # for Gemini Models

vertexai.init(project = PROJECT_ID, location = REGION)

In [ ]:
# Gemini Models
gemini_text = vertexai.generative_models.GenerativeModel("gemini-1.0-pro")
gemini_multimodal = vertexai.generative_models.GenerativeModel("gemini-1.0-pro-vision")

# PaLM and Codey Models
text_model = vertexai.language_models.TextGenerationModel.from_pretrained('text-bison')
chat_model = vertexai.language_models.ChatModel.from_pretrained('chat-bison')
textembed_model = vertexai.language_models.TextEmbeddingModel.from_pretrained('textembedding-gecko')
codegen_model = vertexai.language_models.CodeGenerationModel.from_pretrained('code-bison')
codecomp_model = vertexai.language_models.CodeGenerationModel.from_pretrained('code-gecko')
codechat_model = vertexai.language_models.CodeChatModel.from_pretrained('codechat-bison')

# Imagen Models
imagecap_model = vertexai.vision_models.ImageCaptioningModel.from_pretrained("imagetext")
imageqna_model = vertexai.vision_models.ImageQnAModel.from_pretrained("imagetext")
imagetext_model = vertexai.vision_models.ImageTextModel.from_pretrained("imagetext")
multimodalembed_model = vertexai.vision_models.MultiModalEmbeddingModel.from_pretrained('multimodalembedding')
imagen1_model = vertexai.preview.vision_models.ImageGenerationModel.from_pretrained('imagegeneration@002')
imagen2_model = vertexai.preview.vision_models.ImageGenerationModel.from_pretrained('imagegeneration@005')

In [ ]:
prediction = textgen_model.predict('Create is a short summary of the rules of baseball?')

print(prediction.text)

In [ ]:
prediction = gemini_text.generate_content('Create is a short summary of the rules of baseball?')

print(prediction.text)

---
## Colab Setup

To run this notebook in Colab click [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/Applied%20GenAI/Vertex%20AI%20GenAI%20Embeddings.ipynb) and run the cells in this section.  Otherwise, skip this section.

This cell will authenticate to GCP (follow prompts in the popup).

In [1]:
PROJECT_ID = 'statmike-mlops-349915' # replace with project ID

In [2]:
try:
    import google.colab
    from google.colab import auth
    auth.authenticate_user()
    !gcloud config set project {PROJECT_ID}
except Exception:
    pass

---
## Installs

The list `packages` contains tuples of package import names and install names.  If the import name is not found then the install name is used to install quitely for the current user.

In [ ]:
# tuples of (import name, install name, min_version)
packages = [
    ('google.cloud.aiplatform', 'google-cloud-aiplatform', '1.38.0'),
    ('google.cloud.discoveryengine', 'google-cloud-discoveryengine')
]

import importlib
install = False
for package in packages:
    if not importlib.util.find_spec(package[0]):
        print(f'installing package {package[1]}')
        install = True
        !pip install {package[1]} -U -q --user
    elif len(package) == 3:
        if importlib.metadata.version(package[0]) < package[2]:
            print(f'updating package {package[1]}')
            install = True
            !pip install {package[1]} -U -q --user

## API Enablement

In [ ]:
!gcloud services enable aiplatform.googleapis.com

### Restart Kernel (If Installs Occured)

After a kernel restart the code submission can start with the next cell after this one.

In [2]:
if install:
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

---
## Setup

Inputs

In [1]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [2]:
REGION = 'us-central1'
EXPERIMENT = 'embeddings'
SERIES = 'applied-genai'

# specify a GCS Bucket
GCS_BUCKET = PROJECT_ID

Packages

In [229]:
import vertexai.language_models
import vertexai.vision_models
from google.cloud import bigquery

import os
import pygbif
import requests
import asyncio
import multiprocessing
import tqdm
import time
import IPython.display 

Clients

In [4]:
# vertex ai clients
vertexai.init(project = PROJECT_ID, location = REGION)

# bigquery client
bq = bigquery.Client(project = PROJECT_ID)

parameters:

In [205]:
DIR = f"temp/{EXPERIMENT}"

In [10]:
SERVICE_ACCOUNT = !gcloud config list --format='value(core.account)' 
SERVICE_ACCOUNT = SERVICE_ACCOUNT[0]
SERVICE_ACCOUNT

'1026793852137-compute@developer.gserviceaccount.com'

List the service accounts current roles:

In [11]:
!gcloud projects get-iam-policy $PROJECT_ID --filter="bindings.members:$SERVICE_ACCOUNT" --format='table(bindings.role)' --flatten="bindings[].members"

ROLE
roles/bigquery.admin
roles/owner
roles/run.admin
roles/secretmanager.secretAccessor
roles/storage.objectAdmin


environment:

In [211]:
if not os.path.exists(DIR):
    os.makedirs(DIR)